In [2]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])
import math

# Import the backtrader platform
import backtrader as bt

In [3]:
import pandas as pd
import numpy as np

from sklearn import linear_model as lm
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
%matplotlib inline

# Linear Regression

$$VIX Return \,\,\tilde{ }\,\, Diff_{SMA} + Overbought + Oversold + Golden + Death$$

Key params:

* training data size : 200
* lag : [1, 5, 10, 20]

In [29]:
# Create a Stratey
class LMStrategy(bt.Strategy):
    

    def log(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.dataclose_spx = self.datas[1].close

        
        # Add indicators
        self.sma_short = bt.indicators.SimpleMovingAverage(self.datas[0], period=50)
        self.sma_long = bt.indicators.SimpleMovingAverage(self.datas[0], period=200)
        
        # Add X variables
        self.dsma = self.sma_short - self.sma_long
        self.mom = bt.Cmp(self.sma_short, self.sma_long)
        
        self.rsi_overbought = bt.indicators.RSI(self.datas[0]) > 70
        self.rsi_oversold = bt.indicators.RSI(self.datas[0]) <30
        
        self.cross_up = bt.indicators.CrossUp(self.sma_short, self.sma_long) 
        self.cross_down = bt.indicators.CrossDown(self.sma_short, self.sma_long)
        
        # Compute daily return
        self.vix_returns = self.dataclose / self.dataclose(-1) - 1
        

    def next(self):
        
        # Set the length of training set
        trainsize = 100
        lag = 20
        count = len(self.vix_returns) - 200
        
        self.log('VIX Return, %.4f' % self.vix_returns[0])
        
        # Regression
        
        trainX1 = pd.DataFrame(self.mom.get(ago=-lag, size=trainsize).tolist())
        trainX2 = pd.DataFrame(self.rsi_overbought.get(ago=-lag, size=trainsize).tolist())
        trainX3 = pd.DataFrame(self.rsi_oversold.get(ago=-lag, size=trainsize).tolist())
        trainX4 = pd.DataFrame(self.cross_up.get(ago=-lag, size=trainsize).tolist())
        trainX5 = pd.DataFrame(self.cross_down.get(ago=-lag, size=trainsize).tolist())
        
        trainX = pd.concat([trainX1, trainX2, trainX3, trainX4, trainX5], axis=1)
        
        trainY = np.array(self.vix_returns.get(ago=0, size=trainsize).tolist()).reshape(-1, 1)
        
        testX = np.array([self.mom[1-lag], self.rsi_overbought[1-lag], self.rsi_oversold[1-lag], self.cross_up[1-lag], self.cross_down[1-lag]]).reshape(-1,5)
        testY = self.vix_returns[0]
        
        mse = []
        r_pred = []
        
        if count <= trainsize + lag - 1:
            
            # Set the predicted return to 0 if no regression is running
            r_pred.append(0)
            
        else:
            
            regr = lm.LinearRegression(fit_intercept=True, normalize=True)
            regr.fit(trainX, trainY)
            tmp_pred = regr.predict(testX)
            r_pred.append(tmp_pred) # use current data to predict
            mse.append(math.pow((tmp_pred-testY), 2))
            
        print('Predicted return, %.4f' % r_pred[-1])
        print('Mean Squared Error, %.4f' % np.nanmean(mse))
         
        # Check if we are in the market
        #if not self.position:

            # Not yet ... we MIGHT BUY if the predicted return > 0
        if r_pred[-1] > 0:

                # BUY, BUY, BUY!!! (with all possible default parameters)
            self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
            self.order = self.buy()

        #else:

        if r_pred[-1] < 0:
                # SELL, SELL, SELL!!! (with all possible default parameters)
            self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
            self.order = self.sell()

In [30]:
if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()

    # Add a strategy
    cerebro.addstrategy(LMStrategy)
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='mySharpe')

    datapath_spx = os.path.join('../../../datas/spx-2013-2018.txt')
    datapath_vix= os.path.join('../../../datas/vix-2013-2018.txt')

    # Create a Data Feed
    data_vix = bt.feeds.YahooFinanceCSVData(
        dataname=datapath_vix,
        # Do not pass values before this date
        #fromdate=datetime.datetime(2018, 1, 1),
        # Do not pass values before this date
        #todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    data_spx = bt.feeds.YahooFinanceCSVData(
        dataname=datapath_spx,
        # Do not pass values before this date
        #fromdate=datetime.datetime(2018, 1, 1),
        # Do not pass values before this date
        #todate=datetime.datetime(2018, 2, 9),
        # Do not pass values after this date
        reverse=False)

    
    # Add the Data Feed to Cerebro
    cerebro.adddata(data_vix)
    cerebro.adddata(data_spx)

    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='lm_result_20.csv',csv=True)
    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    # Print out the final result
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    print('Sharpe Ratio:', thestrat.analyzers.mySharpe.get_analysis())

Starting Portfolio Value: 100000.00
2013-11-22, VIX Return, -0.0316
Predicted return, 0.0000
Mean Squared Error, nan
2013-11-25, VIX Return, 0.0432
Predicted return, 0.0000
Mean Squared Error, nan
2013-11-26, VIX Return, 0.0016
Predicted return, 0.0000
Mean Squared Error, nan
2013-11-27, VIX Return, 0.0133
Predicted return, 0.0000
Mean Squared Error, nan
2013-11-29, VIX Return, 0.0555
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-02, VIX Return, 0.0387
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-03, VIX Return, 0.0225
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-04, VIX Return, 0.0103
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-05, VIX Return, 0.0259
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-06, VIX Return, -0.0855
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-09, VIX Return, -0.0218
Predicted return, 0.0000
Mean Squared Error, nan
2013-12-10, VIX Return, 0.0311
Predicted return, 0.0000
Mean Squared Error, nan
2

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:75: RuntimeWarning: Mean of empty slice



Mean Squared Error, nan
2014-02-24, VIX Return, -0.0307
Predicted return, 0.0000
Mean Squared Error, nan
2014-02-25, VIX Return, -0.0394
Predicted return, 0.0000
Mean Squared Error, nan
2014-02-26, VIX Return, 0.0497
Predicted return, 0.0000
Mean Squared Error, nan
2014-02-27, VIX Return, -0.0216
Predicted return, 0.0000
Mean Squared Error, nan
2014-02-28, VIX Return, -0.0028
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-03, VIX Return, 0.1429
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-04, VIX Return, -0.1188
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-05, VIX Return, -0.0149
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-06, VIX Return, 0.0230
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-07, VIX Return, -0.0070
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-10, VIX Return, 0.0064
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-11, VIX Return, 0.0423
Predicted return, 0.0000
Mean Squared Error, nan
2014-03-

Predicted return, 0.0051
Mean Squared Error, 0.0041
2014-07-23, BUY CREATE, 11.52
2014-07-24, VIX Return, 0.0278
Predicted return, 0.0026
Mean Squared Error, 0.0006
2014-07-24, BUY CREATE, 11.84
2014-07-25, VIX Return, 0.0718
Predicted return, 0.0040
Mean Squared Error, 0.0046
2014-07-25, BUY CREATE, 12.69
2014-07-28, VIX Return, -0.0102
Predicted return, 0.0041
Mean Squared Error, 0.0002
2014-07-28, BUY CREATE, 12.56
2014-07-29, VIX Return, 0.0573
Predicted return, 0.0049
Mean Squared Error, 0.0028
2014-07-29, BUY CREATE, 13.28
2014-07-30, VIX Return, 0.0038
Predicted return, 0.0051
Mean Squared Error, 0.0000
2014-07-30, BUY CREATE, 13.33
2014-07-31, VIX Return, 0.2716
Predicted return, 0.0106
Mean Squared Error, 0.0681
2014-07-31, BUY CREATE, 16.95
2014-08-01, VIX Return, 0.0047
Predicted return, 0.0105
Mean Squared Error, 0.0000
2014-08-01, BUY CREATE, 17.03
2014-08-04, VIX Return, -0.1122
Predicted return, 0.0081
Mean Squared Error, 0.0145
2014-08-04, BUY CREATE, 15.12
2014-08-05, 

2014-12-23, VIX Return, -0.0295
Predicted return, 0.0028
Mean Squared Error, 0.0010
2014-12-23, BUY CREATE, 14.80
2014-12-24, VIX Return, -0.0291
Predicted return, 0.0019
Mean Squared Error, 0.0010
2014-12-24, BUY CREATE, 14.37
2014-12-26, VIX Return, 0.0090
Predicted return, 0.0021
Mean Squared Error, 0.0000
2014-12-26, BUY CREATE, 14.50
2014-12-29, VIX Return, 0.0386
Predicted return, 0.0031
Mean Squared Error, 0.0013
2014-12-29, BUY CREATE, 15.06
2014-12-30, VIX Return, 0.0571
Predicted return, 0.0045
Mean Squared Error, 0.0028
2014-12-30, BUY CREATE, 15.92
2014-12-31, VIX Return, 0.2060
Predicted return, 0.0097
Mean Squared Error, 0.0385
2014-12-31, BUY CREATE, 19.20
2015-01-02, VIX Return, -0.0734
Predicted return, 0.0076
Mean Squared Error, 0.0066
2015-01-02, BUY CREATE, 17.79
2015-01-05, VIX Return, 0.1197
Predicted return, 0.0104
Mean Squared Error, 0.0120
2015-01-05, BUY CREATE, 19.92
2015-01-06, VIX Return, 0.0602
Predicted return, 0.0115
Mean Squared Error, 0.0024
2015-01-06

...

Predicted return, 0.0033
Mean Squared Error, 0.0001
2016-08-25, BUY CREATE, 13.63
2016-08-26, VIX Return, 0.0015
Predicted return, 0.0032
Mean Squared Error, 0.0000
2016-08-26, BUY CREATE, 13.65
2016-08-29, VIX Return, -0.0520
Predicted return, 0.0025
Mean Squared Error, 0.0030
2016-08-29, BUY CREATE, 12.94
2016-08-30, VIX Return, 0.0139
Predicted return, 0.0027
Mean Squared Error, 0.0001
2016-08-30, BUY CREATE, 13.12
2016-08-31, VIX Return, 0.0229
Predicted return, 0.0029
Mean Squared Error, 0.0004
2016-08-31, BUY CREATE, 13.42
2016-09-01, VIX Return, 0.0045
Predicted return, 0.0029
Mean Squared Error, 0.0000
2016-09-01, BUY CREATE, 13.48
2016-09-02, VIX Return, -0.1113
Predicted return, 0.0015
Mean Squared Error, 0.0127
2016-09-02, BUY CREATE, 11.98
2016-09-06, VIX Return, 0.0033
Predicted return, 0.0015
Mean Squared Error, 0.0000
2016-09-06, BUY CREATE, 12.02
2016-09-07, VIX Return, -0.0067
Predicted return, 0.0014
Mean Squared Error, 0.0001
2016-09-07, BUY CREATE, 11.94
2016-09-08,

Predicted return, 0.0046
Mean Squared Error, 0.0009
2016-12-08, BUY CREATE, 12.64
2016-12-09, VIX Return, -0.0704
Predicted return, 0.0040
Mean Squared Error, 0.0055
2016-12-09, BUY CREATE, 11.75
2016-12-12, VIX Return, 0.0757
Predicted return, 0.0039
Mean Squared Error, 0.0052
2016-12-12, BUY CREATE, 12.64
2016-12-13, VIX Return, 0.0063
Predicted return, 0.0046
Mean Squared Error, 0.0000
2016-12-13, BUY CREATE, 12.72
2016-12-14, VIX Return, 0.0369
Predicted return, 0.0042
Mean Squared Error, 0.0011
2016-12-14, BUY CREATE, 13.19
2016-12-15, VIX Return, -0.0303
Predicted return, 0.0038
Mean Squared Error, 0.0012
2016-12-15, BUY CREATE, 12.79
2016-12-16, VIX Return, -0.0461
Predicted return, 0.0035
Mean Squared Error, 0.0025
2016-12-16, BUY CREATE, 12.20
2016-12-19, VIX Return, -0.0402
Predicted return, 0.0032
Mean Squared Error, 0.0019
2016-12-19, BUY CREATE, 11.71
2016-12-20, VIX Return, -0.0222
Predicted return, 0.0036
Mean Squared Error, 0.0007
2016-12-20, BUY CREATE, 11.45
2016-12-2

Predicted return, 0.0008
Mean Squared Error, 0.0016
2017-04-06, BUY CREATE, 12.39
2017-04-07, VIX Return, 0.0387
Predicted return, 0.0016
Mean Squared Error, 0.0014
2017-04-07, BUY CREATE, 12.87
2017-04-10, VIX Return, 0.0917
Predicted return, 0.0023
Mean Squared Error, 0.0080
2017-04-10, BUY CREATE, 14.05
2017-04-11, VIX Return, 0.0726
Predicted return, 0.0038
Mean Squared Error, 0.0047
2017-04-11, BUY CREATE, 15.07
2017-04-12, VIX Return, 0.0464
Predicted return, 0.0040
Mean Squared Error, 0.0018
2017-04-12, BUY CREATE, 15.77
2017-04-13, VIX Return, 0.0120
Predicted return, 0.0044
Mean Squared Error, 0.0001
2017-04-13, BUY CREATE, 15.96
2017-04-17, VIX Return, -0.0815
Predicted return, 0.0040
Mean Squared Error, 0.0073
2017-04-17, BUY CREATE, 14.66
2017-04-18, VIX Return, -0.0164
Predicted return, 0.0041
Mean Squared Error, 0.0004
2017-04-18, BUY CREATE, 14.42
2017-04-19, VIX Return, 0.0354
Predicted return, 0.0045
Mean Squared Error, 0.0010
2017-04-19, BUY CREATE, 14.93
2017-04-20, 

Predicted return, 0.0024
Mean Squared Error, 0.0019
2017-08-30, BUY CREATE, 11.22
2017-08-31, VIX Return, -0.0561
Predicted return, 0.0008
Mean Squared Error, 0.0032
2017-08-31, BUY CREATE, 10.59
2017-09-01, VIX Return, -0.0434
Predicted return, -0.0004
Mean Squared Error, 0.0019
2017-09-01, SELL CREATE, 10.13
2017-09-05, VIX Return, 0.2073
Predicted return, 0.0013
Mean Squared Error, 0.0424
2017-09-05, BUY CREATE, 12.23
2017-09-06, VIX Return, -0.0491
Predicted return, 0.0007
Mean Squared Error, 0.0025
2017-09-06, BUY CREATE, 11.63
2017-09-07, VIX Return, -0.0069
Predicted return, 0.0146
Mean Squared Error, 0.0005
2017-09-07, BUY CREATE, 11.55
2017-09-08, VIX Return, 0.0494
Predicted return, 0.0233
Mean Squared Error, 0.0007
2017-09-08, BUY CREATE, 12.12
2017-09-11, VIX Return, -0.1147
Predicted return, 0.0013
Mean Squared Error, 0.0134
2017-09-11, BUY CREATE, 10.73
2017-09-12, VIX Return, -0.0140
Predicted return, 0.0017
Mean Squared Error, 0.0002
2017-09-12, BUY CREATE, 10.58
2017-0

Predicted return, 0.0016
Mean Squared Error, 0.0005
2018-01-22, BUY CREATE, 11.03
2018-01-23, VIX Return, 0.0063
Predicted return, 0.0013
Mean Squared Error, 0.0000
2018-01-23, BUY CREATE, 11.10
2018-01-24, VIX Return, 0.0333
Predicted return, 0.0021
Mean Squared Error, 0.0010
2018-01-24, BUY CREATE, 11.47
2018-01-25, VIX Return, 0.0096
Predicted return, 0.0028
Mean Squared Error, 0.0000
2018-01-25, BUY CREATE, 11.58
2018-01-26, VIX Return, -0.0432
Predicted return, 0.0028
Mean Squared Error, 0.0021
2018-01-26, BUY CREATE, 11.08
2018-01-29, VIX Return, 0.2491
Predicted return, 0.0032
Mean Squared Error, 0.0605
2018-01-29, BUY CREATE, 13.84
2018-01-30, VIX Return, 0.0686
Predicted return, 0.0044
Mean Squared Error, 0.0041
2018-01-30, BUY CREATE, 14.79
2018-01-31, VIX Return, -0.0845
Predicted return, 0.0036
Mean Squared Error, 0.0078
2018-01-31, BUY CREATE, 13.54
2018-02-01, VIX Return, -0.0052
Predicted return, 0.0035
Mean Squared Error, 0.0001
2018-02-01, BUY CREATE, 13.47
2018-02-02,

Linear regression (Lag = 1):

Mean Squared Error, 0.0440

Final Portfolio Value: 108715.71


Lag = 5:

Mean Squared Error, NAN

Final Portfolio Value: 106643.49


Lag = 10:

Mean Squared Error, 0.0219

Final Portfolio Value: 106126.81


Lag = 20:

Mean Squared Error, 0.0219

Final Portfolio Value: 107526.55